In [1]:
import csv
from mip import * # Model, xsum, minimize, BINARY, OptimizationStatus
from datetime import datetime

In [3]:
liczba_M = 9999 # duża liczba (większa, niż możliwy do przejechania dystans podczas jednego kursu)
separatorCSV = ";" # separator kolumn w pliku CSV

# folder z którego są pobierane dane i do którego są zapisywane wyniki
folder = "./data/"
plik_nazwa = "stacjeR04_07_01"
nazwa_pliku_stacje = folder + plik_nazwa + ".csv" # nazwa pliku z popytem na stacjach
nazwa_pliku_odl = folder + "odlR04.csv" # nazwa pliku z odległościami pomiędzy stacjami
nazwa_pliku_wyniki = folder + plik_nazwa + "_GRB_2.csv"

f = open(nazwa_pliku_wyniki, 'w')

# tworzy "pusty" model programowania liniowego
model_VRPTW1 = Model("VRPTW_LPG3dni", solver_name=GRB) # CBC
liczba_dni = 3 # liczba dni, dla których ustala się trasy (na razie nie można zmieniać)
KOLUMNA_MOZE = 8 # nr pierwszej kolumny z informację, które stacje mogą być odwiedzone
KOLUMNA_MUSI = 11 # nr pierwszej kolumny z informację, które stacje muszą być odwiedzone

with open(nazwa_pliku_stacje) as csvfile:
    plik = csv.reader(csvfile, delimiter=separatorCSV)
    dane = list(plik)

# usuwa pierwszy wiersz z nazwami kolumn (jeśli istnieje)
try: int(dane[0][1])
except: del dane[0]

# szuka wiersza z rozlewnią (rozlewnia ma ujemny popyt) i usuwa go
usun = -1

for i in range(len(dane)):
    if int(dane[i][1]) < 0: usun = i
if usun >= 0:
    rozlewnia = list(dane[usun])
    del dane[usun]
else:
    print("Brak danych o rozlewni w pliku ", nazwa_pliku_stacje)
    # break
ile_stacji = len(dane)

# dodaje rozlewnię na początku i na końcu listy
dane.insert(0, rozlewnia)
dane.insert(len(dane), rozlewnia)

# łączy dane z kolumn w tuple
dane = list(zip(*dane))

# zamienia separator części dziesiętnej, jeśli w pliku CSV użyto ','
dane[4] = [znak.replace(",", ".") for znak in dane[4]]
dane[5] = [znak.replace(",", ".") for znak in dane[5]]
dane[6] = [znak.replace(",", ".") for znak in dane[6]]
dane[16] = [znak.replace(",", ".") for znak in dane[16]]
dane[17] = [znak.replace(",", ".") for znak in dane[17]]
dane[18] = [znak.replace(",", ".") for znak in dane[18]]
dane[19] = [znak.replace(",", ".") for znak in dane[19]]
dane[20] = [znak.replace(",", ".") for znak in dane[20]]

nazwa = list(dane[0]) # nazwy (unikalne symbole) stacji benzynowych i rozlewni

ile_stacji = len(nazwa) - 2

with open(nazwa_pliku_odl) as csvfile:
    plik2 = csv.reader(csvfile, delimiter=separatorCSV)
    dane2  = list(plik2)

# usuwa pierwszy wiersz z nazwami kolumn (jeśli istnieje)
try: int(dane2[0][2])
except: del dane2[0]

# tworzy macierz odległości i macierz czasu przejazdu i wypełnia je zerami
macierz_odl=[]
macierz_czas=[]
for i in range(ile_stacji+1):
    macierz_odl.append([])
    macierz_czas.append([])

macierz_odl=[[0 for j in range(ile_stacji+2)] for i in range(ile_stacji+1)]
macierz_czas=[[0 for j in range(ile_stacji+2)] for i in range(ile_stacji+1)]

# przepisuje odległości i czasy przejazdu z pliku CSV do macierzy: macierz_odl i macierz_czas
for i in range(len(dane2)):
    if (dane2[i][0] in nazwa) and (dane2[i][1] in nazwa):
        macierz_odl[nazwa.index(dane2[i][0])][nazwa.index(dane2[i][1])]=float(dane2[i][2].replace(",", "."))
        macierz_czas[nazwa.index(dane2[i][0])][nazwa.index(dane2[i][1])]=float(dane2[i][3].replace(",", "."))

# rozlewnia jest w pierwszym wierszu macierz oraz w pierwszej i ostatniej kolumnie macierzy
# kopiuje zawartość pierwszej kolumny do ostatniej kolumny
for i in range(ile_stacji+1):
    macierz_odl[i][ile_stacji + 1] = macierz_odl[i][0]
    macierz_czas[i][ile_stacji + 1] = macierz_czas[i][0]

# pobiera współrzędne GPS stacji    
GPSx = tuple(map(float, dane[19]))
GPSy = tuple(map(float, dane[20]))

# parametry
par_K = 0 # liczba wyznaczanych tras
par_N = [] # liczba tras wyznaczanych w określonym dniu
for i in range(liczba_dni):
    par_N.append(int(dane[KOLUMNA_MOZE + i][0]))
    par_K += par_N[-1]

par_Dzien = [] # numery dni dla których wyznaczana jest kolejna trasa
par_Dzien2 = []
k = 0
for i in range(liczba_dni):
    dni_tmp = []
    for j in range(par_N[i]):
        par_Dzien2.append(i)
        dni_tmp.append(k)
        k += 1
    par_Dzien.append(dni_tmp)

par_I = ile_stacji # liczba stacji benzynowych [int]

par_B = -int(rozlewnia[1]) # pojemność cysterny [int]

par_D = macierz_odl # macierz odległości [float]

par_T = macierz_czas # macierz czasów przejazdu [float]

par_PI = list(map(int,dane[2*liczba_dni+1])) # maksymalna liczba odwiedzin danej stacji w badanym okresie

par_PM = [] # stacje które można zatankować w określonym dniu [bin]
for i in range(liczba_dni):
    par_PM.append(list(map(int,dane[KOLUMNA_MOZE+i])))

par_PK = [] # stacje pilne (muszą być zatankowane w określonym dniu) [bin]
for i in range(liczba_dni):
    par_PK.append(list(map(int,dane[KOLUMNA_MUSI+i])))

par_PK_mix = [] # stacje pilne (muszą być zatankowane w określonym dniu albo dzień wcześniej) [bin]
for i in range(liczba_dni-1):
    par_PK_mix.append(list(map(int,dane[KOLUMNA_MUSI+liczba_dni+i])))

par_C = [] # popyt na stacjach [int]
for i in range(liczba_dni):
    par_C.append(list(map(int,dane[i+1])))

par_S = [] # czas obsługi na stacji (service time) [float]
for i in range(liczba_dni):
    par_S.append(list(map(float, dane[i + liczba_dni + 1])))

par_V = [] # najpóżniejszy czas przyjzadu na stacje w określonym dniu (dotyczy tylko stacji pilnych) [float]
for i in range(liczba_dni):
    par_V.append(list(map(float, dane[KOLUMNA_MUSI+liczba_dni*2+i-1])))
    for j in range(ile_stacji+2):
        if par_V[i][j] == 0:
            par_V[i][j] = 999 # jeśli nie określono czasu przyjazdu, to jest on duży

for idx in range(10):
    czas_obliczen = 160 + 10 * idx  # maksymalny czas obliczeń (w sekundach)
    # kasuje zawartość modelu VRPTW
    model_VRPTW1.clear()
    
    # zmienne decyzyjne

    # opis zmiennych zm_x
    # zm_x[i][j] - przejazd cysterny z i-tej do j-tej stacji
    # zm_x[i][j]=1 jeśli cysterna ma pojechać z i-tej do j-tej stacji
    # zm_x[i][j]=0 jeśli cysterna ma nie jedzie z i-tej do j-tej stacji
    # i=0,1,...,par_I;   j=0,1,...,par_I
    # i=0 lub j=par_I -> rozlewnia
    # czyli stacje paliw dla i=1,...,par_I lub dla j=0,...,par_I-1 (stacja n: dla i=n lub dla j=n-1)!!!
    zm_x = [[[model_VRPTW1.add_var(var_type=BINARY) for k in range(par_K)]
            for j in range(par_I+1)]
            for i in range(par_I+1)]

    # opis zmiennych zm_y
    # zm_y[j] - czas przyjazdu na j-tą stację (jeśli stacja jest na trasie)
    # j=par_I -> powrót na rozlewnię (koniec kursu)
    zm_y = [[model_VRPTW1.add_var() for k in range(par_K)] for j in range(par_I+2)]

    # funkcja celu (minimalizacja odległości)
    model_VRPTW1.objective = minimize(xsum(par_D[i][j+1]*zm_x[i][j][k] for i in range(par_I+1) for j in range(par_I+1) for k in range(par_K)))

    # ograniczenia

    # 1. Cysterna może pojechać tylko na stacje, które uwzględniono w wektorze par_PM dla danego dnia
    for dz in range(liczba_dni):
        for j in range(par_I):
            model_VRPTW1 += xsum(zm_x[i][j][k] for k in par_Dzien[dz] for i in range(par_I+1)) <= par_PM[dz][j+1]

    # 2. Cysterna rozpoczyna trasę w rozlewni
    for k in range(par_K):
        model_VRPTW1 += xsum(zm_x[0][j][k] for j in range(par_I + 1)) == 1

    # 3. Cysterna kończy trasę w rozlewni
    for k in range(par_K):
        model_VRPTW1 += xsum(zm_x[i][par_I][k] for i in range(par_I + 1)) == 1

    # 4. Cysterna po przyjechaniu na i-tą stację musi z niej odjechać
    for i in range(par_I):
        for k in range(par_K):
            model_VRPTW1 += xsum(zm_x[j][i][k] - zm_x[i + 1][j][k]
                                for j in range(par_I + 1)) == 0

    # 5. Ograniczenia ustalające czasy przyjazdu cysterny na stacje benzynowe
    for i in range(par_I + 1):
        for j in range(par_I + 1):
            for k in range(par_K):
                if i!=j+1:
                    model_VRPTW1 += (zm_y[i][k] - zm_y[j+1][k] + par_S[par_Dzien2[k]][i] + par_T[i][j+1] - (1 - zm_x[i][j][k]) * liczba_M) <= 0

    # 6. Ustalenie maksymalnego czasu przyjazdu na pilne stacje benzynowe i maksymalnego czasu dla całej trasy
    for i in range(par_I + 2):
        for k in range(par_K):
            model_VRPTW1 += zm_y[i][k] <= par_V[par_Dzien2[k]][i]

    # 7. Cysterna jest całkowicie opróżniana (popyt ostatniej stacji na trasie nie musi być w pełni zaspokojony)
    for k in range(par_K):
        model_VRPTW1 += xsum(zm_x[i][j][k] * par_C[par_Dzien2[k]][i]
                            for i in range(1, par_I + 1)
                            for j in range(par_I + 1)) >= par_B

    # 8. Suma popytu na odwiedzanych stacjach (oprócz ostatniej) jest mniejsza niż pojemność cysterny
    for k in range(par_K):
        model_VRPTW1 += xsum(zm_x[i][j][k] * par_C[par_Dzien2[k]][i] 
                            for i in range(1, par_I + 1)
                            for j in range(par_I)) <= par_B

    # 9. Cysterna musi pojechać na stacje priorytetowe (pilne), które uwzględniono w wektorze par_PK - ostatnia odwiedzana stacja nie jest priorytetowa, dlatego j < par_I
    for i in range(1, par_I+1):
        for dz in range(liczba_dni):
            model_VRPTW1 += xsum(zm_x[i][j][k] for j in range(par_I) for k in par_Dzien[dz]) >= par_PK[dz][i]

    # 10. Cysterna musi pojechać na stacje priorytetowe (pilne), które uwzględniono w wektorze par_PK_mix w danym dniu lub dzień wcześniej
    for i in range(1, par_I + 1):
        for dz in range(liczba_dni - 1):
            model_VRPTW1 += xsum(zm_x[i][j][k] 
                                for j in range(par_I) 
                                for k in par_Dzien[dz] + par_Dzien[dz + 1]) >= par_PK_mix[dz][i]

    # 11. Nie można jechać ze stacji do tej samej stacji
    for k in range(par_K):
        for i in range(par_I):
            model_VRPTW1 += zm_x[i+1][i][k] == 0
        model_VRPTW1 += zm_x[0][par_I][k] == 0

    # 12. Czas przyjazdu na rozlewnię (początek kursu)
    for k in range(par_K):
        zm_y[0][k]=0.0

    # 13. Każda stacja może być odwiedzona tylko tyle razy ile zapisano w wektorze par_PI
    for j in range(par_I):
        model_VRPTW1 += xsum(zm_x[i][j][k] for k in range(par_K) for i in range(par_I+1)) <= par_PI[j+1]
    
    czas_start = datetime.now()
    # model_VRPTW1.max_mip_gap = 0.03
    wynik = model_VRPTW1.optimize(max_seconds=czas_obliczen)

    czas_obliczenia = datetime.now() - czas_start
    
    print(wynik)
    print(f"{model_VRPTW1.objective_value}\t{czas_obliczenia.seconds}\n")
    f.write(f"{model_VRPTW1.objective_value}\t{czas_obliczenia.seconds}\n")
    
    # if (wynik == OptimizationStatus.OPTIMAL) or (wynik == OptimizationStatus.FEASIBLE):
    #     # trasa = []

f.close()
        

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-28
Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-28
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter TimeLimit to value 160
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter Cuts to value 1
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 22376 rows, 21455 columns and 231294 nonzeros
Model fingerprint: 0x85458678
Variable types: 330 continuous, 21125 integer (21125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 19729 rows and 18880 columns
Presol